# Prefect Workers and Queues

### Introduction

In the last lesson, we learned about setting up a deployment with prefect, and we saw how our deployment is associated with a particular flow, and our deployment schedules flow runs.

In this lesson, we'll look a little bit deeper at that deployment.  As we'll see our deployment often creates a queue, and there will be a worker pool will allocate a worker to pull items off the queue. 

### Looking again at our deployment

If we run our current code -- `python3 index.py`, and then look at our prefect dashboard, we can see our the work that is scheduled to be performed.

<img src="./queue-runs.png">

This is a queue.  The queue stores the scheduled workflow-runs that are waiting to be run.  They're just stored in a table on prefect.

To *perform* the work, prefect assigns the queue of work to a workpool, which contains *workers*, labeled 1 through 4 below.  

<img src="./work-queue.png" width="40%">

> Above you can see our scheduled workflows (items 1 - 3) in the queue.  And the pool of workers allocated for performing that work.

At the scheduled time, a worker will be assigned the item, and execute the tasks in the workflow.

<img src="./worker-assigned.png" width="60%">

> Above you can see the worker 1 is now executing item 1.

Where is this workpool in prefect?  Well in the prefect UI, you can see a representation of the workpool, if you click on the workpools tab.

<img src="./prefect-workpool.png" width="90%">

But really, the work pool will live on our computer.  

In other, words prefect will manage the *orchestration environment* on the cloud -- but we will have to pay for (or provide) the execution environment.  

> If we were to deploy our prefect code to an AWS computer, then the execution environment would by default be that AWS computer.

### Moving to Prefect

Ok, so now we've learned that when we create a deployment, prefect will create a **queue** of workflow-runs to be performed, and a **worker pool** that assigns workers to execute them.  Now let's see how we can work with a queue and worker pools in prefect.

### Creating a custom deployment

To specify a queue and worker pool, we'll have to change the way we create deployments.  So in the index.py file, comment out the lines using the get_restaurants.serve, and uncomment the `get_restaurants("HONDURAS MAYA CAFE & BAR LLC")` call.

```python
get_restaurants("HONDURAS MAYA CAFE & BAR LLC")

# if __name__ == "__main__":
#     get_restaurants.serve(
#         name="get-restaurants-deployment",
#         schedule=IntervalSchedule(interval=120),
#         parameters={'url': "HONDURAS MAYA CAFE & BAR LLC"}
#         )
```

And then we can create a deployment by adding the following:

```python

from prefect.server.schemas.schedules import IntervalSchedule
from prefect.deployments.deployments import Deployment
# add the lines above to the top of the index.py file

if __name__ == "__main__":
    schedule = IntervalSchedule(interval=10)
    parameters={'url': "HONDURAS MAYA CAFE & BAR LLC"}
    deployment = Deployment.build_from_flow(
        name="honduras_maya",
        flow=get_restaurants,
        version=1,
        schedule=schedule,
        is_schedule_active=True,
        work_queue_name="default",
        parameters=parameters,
        entrypoint="./index.py:get_restaurants",
    )
    deployment.apply(upload=True)
```

So notice that with the command above, with our deployment we have specified our `worker_queue_name`.  We also, specified:

* name: name of the deployment
* flow: the function that called
* version: the version of the api
* schedule: the schedule active
* the work queue name
* the parameters to pass through
* the location of our flow

### Applying the deployment

At the end of our code above, we have the line:
```python    
deployment.apply(upload=True)
```

This will apply our deployment, once we run `python3 index.py`.

If you boot up the prefect webserver, and click on the deployments tab, you should see the listed deployment.

<img src="./deployment-new.png">

### Starting the Work Pool

The one issue remaining, is that the work pool is not currently running on our computer.  Remember: Prefect will provide the orchestration environment, but we will need to provide the execution environment for the workpool.  

So place the following in the terminal, to start up the prefect pool.

`prefect agent start -p 'default-agent-pool'`

<img src="./workqueue.png">

And also, click on the work pool tab in the prefect UI and make sure it is running.

<img src="./turn-on.png">

Ok, now prefect should be able to take workflow-runs from the queue at the scheduled time and execute them.

### Cleaning up

Finally, if we would like to stop a deployment, we can do so by going to prefect cloud, clicking on deployments and clicking on the button over to the right so that it is no longer green.

<img src="./pause-deployment.png">

Or we can also click on our deployment and then delete it.

<img src="./delete-deploy.png">

### Summary

In this lesson we saw how to schedule a flow with a deployment.  We did so by using our Deployment class.



```python
if __name__ == "__main__":
    schedule = IntervalSchedule(interval=10)
    parameters={'url': "HONDURAS MAYA CAFE & BAR LLC"}
    deployment = Deployment.build_from_flow(
        name="honduras_maya",
        flow=get_restaurants,
        version=1,
        schedule=schedule,
        is_schedule_active=True,
        work_queue_name="default",
        parameters=parameters,
        entrypoint="./index.py:get_restaurants",
    )
    deployment.apply(upload=True)

```

And then applying the deployment by running the script where we defined our deployment.  Finally, we started up a workpool to allocate to running our workflow.

```bash
prefect agent start -p 'default-agent-pool'
```

And then we also made sured that work pool was green in the UI.

<img src="./turn-on.png">